# 📧 Spam Email Classifier (NLP)

**Goal:** Build a machine learning model to classify emails as Spam or Not Spam (Ham) using NLP techniques.

Dataset: [SMS Spam Collection Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)


## 📦 1. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import joblib

try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

## 📁 2. Load Data

In [ ]:
# Replace with your actual data path
df = pd.read_csv('../data/spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'text']
df.head()

## 🧹 3. Preprocessing

In [ ]:
ps = PorterStemmer()

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = ''.join([c for c in text if c not in string.punctuation])
    tokens = text.split()
    tokens = [ps.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(clean_text)
df.head()

## 🔢 4. Encoding Labels

In [ ]:
df['label_num'] = df['label'].map({'ham':0, 'spam':1})
df[['label', 'label_num']].head()

## ✂️ 5. Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['label_num'], test_size=0.2, random_state=42)
print(f"Train size: {len(X_train)} | Test size: {len(X_test)}")

## 🏗️ 6. Vectorization (TF-IDF)

In [ ]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Save vectorizer for later use
joblib.dump(vectorizer, '../models/vectorizer.pkl')

## 🤖 7. Model Training (Naive Bayes)

In [ ]:
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Save model
joblib.dump(model, '../models/spam_model.pkl')

## 🧠 8. Evaluation

In [ ]:
y_pred = model.predict(X_test_vec)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc*100:.2f}%\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ham','Spam'], yticklabels=['Ham','Spam'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## 📝 9. Inference (Predict New Email)

In [ ]:
def predict_spam(text):
    cleaned = clean_text(text)
    vect = vectorizer.transform([cleaned])
    pred = model.predict(vect)[0]
    return 'Spam' if pred == 1 else 'Ham'

# Test
test_email = "Congratulations, you've won a $1000 Walmart gift card. Go to http://bit.ly/123456 to claim now!"
print("Prediction for test email:", predict_spam(test_email))